# Testing

In [ ]:
from pathlib import Path
from src.datasets import TestDataset
from torch.utils.data import DataLoader
from torchvision.transforms import transforms, InterpolationMode
import torch

In [ ]:
IM_RES = 256

In [ ]:
transform = transforms.Compose([
    transforms.Resize((IM_RES, IM_RES), interpolation=InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

In [ ]:
basepath = Path().resolve().parent.parent
labeled_tabs_path = basepath / "data" / "labelled_tabs.csv"
image_path = basepath / "data" / "dataset" / "tabs"
filenames = ["page-9_tab-0.png", "page-9_tab-1.png", "page-466_tab-9.png", "page-466_tab-5.png"]
predict_dataset = TestDataset.from_dir(image_path, filenames=filenames, transform=transform)
predict_loader = DataLoader(predict_dataset, shuffle=False, batch_size=1)

In [ ]:
# TODO: Load fret_pos and finger model weights
fret_pos_model = ...
finger_model = ...

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
fret_pos_model.to(device)
finger_model.to(device)

In [ ]:
# TODO: Annotate more data
# TODO: Split train, val & test sets according to a config file (test on unseen data!)
expected_classes = [
    "[0, 4, 3, 1, 2, 1] | [0, 3, 2, 0, 1, 0]",
    "[2, 4, 4, 3, 2, 2] | [1, 3, 4, 2, 1, 1]",
    "[0, 0, 5, 6, 4, 2] | [0, 0, 3, 4, 2, 1]",
    "[3, 2, 3, 3, 0, 0] | [2, 1, 3, 4, 0, 0]"
]
for im, filename, expected in zip(predict_loader, filenames, expected_classes):
    im = im.to(device)
    frets_classified = fret_pos_model.predict(im).tolist()[0]
    fingers_classified = finger_model.predict(im).tolist()[0]
    print(filename)
    print(f"-> Expected:  {expected}")
    print(f"-> Predicted: {frets_classified} | {fingers_classified}")